In [5]:
NUMBER_TAPS = 10
NUMBER_BITS_COEFS = 8
NUMBER_BITS_SAMPLES = 12
NUMBER_BITS_MULTIPLICATION = 20
# NUMBER BITS ADDITION INCREASE WITH EACH STAGE 

In [24]:
vhdl_components = []
vhdl_header = '''
 LIBRARY IEEE;
 USE IEEE.std_logic_1164.all;
 USE IEEE.numeric_std.ALL;
 
 ENTITY FIR_IAN IS
    PORT( clk                             :   IN    std_logic; 
          clk_enable                      :   IN    std_logic; 
          reset                           :   IN    std_logic; 
          filter_in                       :   IN    std_logic_vector(11 DOWNTO 0); -- sfix16_En15
          filter_out                      :   OUT   std_logic_vector(28 DOWNTO 0)  -- sfix33_En31
          );
 
 END FIR_IAN;
'''
vhdl_architechture_header = '''
 ----------------------------------------------------------------
 --Module Architecture: FIR_IAN
 ----------------------------------------------------------------
 ARCHITECTURE rtl OF FIR_IAN IS
   -- Local Functions
   -- Type Definitions
   TYPE delay_pipeline_type IS ARRAY (NATURAL range <>) OF signed(11 DOWNTO 0); -- sfix16_En15
   -- Constants
   CONSTANT coeff1                         : signed(7 DOWNTO 0) := to_signed(-1, 8); -- sfix16_En16
'''
# COEFS
#lambda expression that returns a string with a template
vhdl_coef_template = lambda index, coef: f"CONSTANT coeff{index}                         : signed({NUMBER_BITS_COEFS-1} DOWNTO 0) := to_signed(REPLACE_ME, {NUMBER_BITS_COEFS});"
vhdl_coefs = [vhdl_coef_template(index, coef) for index, coef in enumerate(range(NUMBER_TAPS))]
#join vhdl_coefs a string separated by a new line character
vhdl_coefs = '\n'.join(vhdl_coefs) 


vhdl_delay_pipeline_template = lambda index: f"signal delay_pipeline : delay_pipeline_type(0 to {NUMBER_TAPS-1});"
# Product template
#lambda expression that returns a string with a template
vhdl_signal_product_template = lambda index, coef: f"SIGNAL product{index}                         : signed({NUMBER_BITS_MULTIPLICATION-1} DOWNTO 0);"
vhdl_signal_product = [vhdl_signal_product_template(index, coef) for index, coef in enumerate(range(NUMBER_TAPS))]
vhdl_signal_product = '\n'.join(vhdl_signal_product) 

vhdl_signal_mul_temp_template = lambda index, coef: f"SIGNAL mul_temp{'' if index==0 else '_'+str(index)}                         : signed({NUMBER_BITS_MULTIPLICATION-1} DOWNTO 0);"
vhdl_signal_mul_temp = [vhdl_signal_mul_temp_template(index, coef) for index, coef in enumerate(range(NUMBER_TAPS))]
vhdl_signal_mul_temp = '\n'.join(vhdl_signal_mul_temp) 

# Sum template
#lambda expression that returns a string with a template
vhdl_signal_sum_template = lambda index, numbits: f"SIGNAL sum{index+1}                             : signed({numbits-1} DOWNTO 0);"
vhdl_signal_sum = [vhdl_signal_sum_template(index, NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+index+1) for index in range(NUMBER_TAPS-1)]

vhdl_signal_add_template = lambda index, numbits: f"SIGNAL add_temp{'' if index==0 else '_'+str(index)}                             : signed({numbits-1} DOWNTO 0);"
vhdl_add_temp_sum = [vhdl_signal_add_template(index, NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+index+1) for index in range(NUMBER_TAPS-1)]

vhdl_signal_sum = '\n'.join(vhdl_signal_sum)
vhdl_add_temp_sum = '\n'.join(vhdl_add_temp_sum)

vhdl_output_typeconvert = f"SIGNAL output_typeconvert               : signed({NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+NUMBER_TAPS-2} DOWNTO 0);"
vhdl_output_register = f"SIGNAL output_typeconvert               : signed({NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+NUMBER_TAPS-2} DOWNTO 0);"

vhdl_delay_pipeline_process = '''
  -- Block Statements
  Delay_Pipeline_process : PROCESS (clk, reset)
  BEGIN
    IF reset = '1' THEN
      delay_pipeline(0 TO 9) <= (OTHERS => (OTHERS => '0'));
    ELSIF clk'event AND clk = '1' THEN
      IF clk_enable = '1' THEN
        delay_pipeline(0) <= signed(filter_in);
        delay_pipeline(1 TO 9) <= delay_pipeline(0 TO 8);
      END IF;
    END IF; 
  END PROCESS Delay_Pipeline_process;
  '''

# Multiplications
line_1_template = lambda index: f"mul_temp{'' if index==0 else '_'+str(index)} <= delay_pipeline({NUMBER_TAPS-1-index}) * coeff{NUMBER_TAPS-index}; \n"
line_2_template = lambda index: f"product{NUMBER_TAPS-1-index} <= resize(mul_temp{'' if index==0 else '_'+str(index)}, {NUMBER_BITS_MULTIPLICATION}); \n"

vhdl_multiplications_both_lines = [line_1_template(index) + line_2_template(index)   for index in range(NUMBER_TAPS)]
vhdl_multiplications_both_lines = "\n".join(vhdl_multiplications_both_lines)

# Sums
temp_lambda = lambda index: f"product{index}" if index == 0 else f"sum{index}" 
line_1_template = lambda index: f"add_temp{'' if index==0 else '_'+str(index)} <= resize({temp_lambda(index)}, {NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+index+1}) + resize(product{index+1}, {NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+index+1}); \n"
line_2_template = lambda index: f"sum{index+1} <= resize(add_temp{'' if index==0 else '_'+str(index)}, {NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+index+1}); \n"

vhdl_sums_both_lines = [line_1_template(index) + line_2_template(index)   for index in range(NUMBER_TAPS-1)]
vhdl_sums_both_lines = "\n".join(vhdl_sums_both_lines)


In [25]:
print(vhdl_sums_both_lines)

add_temp <= resize(product0, 21) + resize(product1, 21); 
sum1 <= resize(add_temp, 21); 

add_temp_1 <= resize(sum1, 22) + resize(product2, 22); 
sum2 <= resize(add_temp_1, 22); 

add_temp_2 <= resize(sum2, 23) + resize(product3, 23); 
sum3 <= resize(add_temp_2, 23); 

add_temp_3 <= resize(sum3, 24) + resize(product4, 24); 
sum4 <= resize(add_temp_3, 24); 

add_temp_4 <= resize(sum4, 25) + resize(product5, 25); 
sum5 <= resize(add_temp_4, 25); 

add_temp_5 <= resize(sum5, 26) + resize(product6, 26); 
sum6 <= resize(add_temp_5, 26); 

add_temp_6 <= resize(sum6, 27) + resize(product7, 27); 
sum7 <= resize(add_temp_6, 27); 

add_temp_7 <= resize(sum7, 28) + resize(product8, 28); 
sum8 <= resize(add_temp_7, 28); 

add_temp_8 <= resize(sum8, 29) + resize(product9, 29); 
sum9 <= resize(add_temp_8, 29); 



In [26]:
print(vhdl_add_temp_sum)

SIGNAL add_temp                             : signed(20 DOWNTO 0);
SIGNAL add_temp_1                             : signed(21 DOWNTO 0);
SIGNAL add_temp_2                             : signed(22 DOWNTO 0);
SIGNAL add_temp_3                             : signed(23 DOWNTO 0);
SIGNAL add_temp_4                             : signed(24 DOWNTO 0);
SIGNAL add_temp_5                             : signed(25 DOWNTO 0);
SIGNAL add_temp_6                             : signed(26 DOWNTO 0);
SIGNAL add_temp_7                             : signed(27 DOWNTO 0);
SIGNAL add_temp_8                             : signed(28 DOWNTO 0);


In [27]:
print(vhdl_signal_product)

SIGNAL product0                         : signed(19 DOWNTO 0);
SIGNAL product1                         : signed(19 DOWNTO 0);
SIGNAL product2                         : signed(19 DOWNTO 0);
SIGNAL product3                         : signed(19 DOWNTO 0);
SIGNAL product4                         : signed(19 DOWNTO 0);
SIGNAL product5                         : signed(19 DOWNTO 0);
SIGNAL product6                         : signed(19 DOWNTO 0);
SIGNAL product7                         : signed(19 DOWNTO 0);
SIGNAL product8                         : signed(19 DOWNTO 0);
SIGNAL product9                         : signed(19 DOWNTO 0);
